The following additional libraries are needed to run this
notebook. Note that running on Colab is experimental, please report a Github
issue if you have any problem.

In [ ]:
!pip install flax
!pip install d2l==1.0.0-beta0


# Custom Layers

One factor behind deep learning's success
is the availability of a wide range of layers
that can be composed in creative ways
to design architectures suitable
for a wide variety of tasks.
For instance, researchers have invented layers
specifically for handling images, text,
looping over sequential data,
and
performing dynamic programming.
Sooner or later, you will need
a layer that does not exist yet in the deep learning framework.
In these cases, you must build a custom layer.
In this section, we show you how.


In [1]:
import jax
from flax import linen as nn
from jax import numpy as jnp
from d2l import jax as d2l

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


## (**Layers without Parameters**)

To start, we construct a custom layer
that does not have any parameters of its own.
This should look familiar if you recall our
introduction to modules in :numref:`sec_model_construction`.
The following `CenteredLayer` class simply
subtracts the mean from its input.
To build it, we simply need to inherit
from the base layer class and implement the forward propagation function.


In [2]:
class CenteredLayer(nn.Module):
    def __call__(self, X):
        return X - X.mean()

Let's verify that our layer works as intended by feeding some data through it.


In [3]:
layer = CenteredLayer()
layer(jnp.array([1.0, 2, 3, 4, 5]))

Array([-2., -1.,  0.,  1.,  2.], dtype=float32)

We can now [**incorporate our layer as a component
in constructing more complex models.**]


In [4]:
net = nn.Sequential([nn.Dense(128), CenteredLayer()])

As an extra sanity check, we can send random data
through the network and check that the mean is in fact 0.
Because we are dealing with floating point numbers,
we may still see a very small nonzero number
due to quantization.


Here we utilize the `init_with_output` method which returns both the output of
the network as well as the parameters. In this case we only focus on the
output.


In [5]:
Y, _ = net.init_with_output(d2l.get_key(), jax.random.uniform(d2l.get_key(),
                                                              (4, 8)))
Y.mean()

Array(-7.450581e-09, dtype=float32)

## [**Layers with Parameters**]

Now that we know how to define simple layers,
let's move on to defining layers with parameters
that can be adjusted through training.
We can use built-in functions to create parameters, which
provide some basic housekeeping functionality.
In particular, they govern access, initialization,
sharing, saving, and loading model parameters.
This way, among other benefits, we will not need to write
custom serialization routines for every custom layer.

Now let's implement our own version of the  fully connected layer.
Recall that this layer requires two parameters,
one to represent the weight and the other for the bias.
In this implementation, we bake in the ReLU activation as a default.
This layer requires two input arguments: `in_units` and `units`, which
denote the number of inputs and outputs, respectively.


In [6]:
class MyDense(nn.Module):
    in_units: int
    units: int

    def setup(self):
        self.weight = self.param('weight', nn.initializers.normal(stddev=1),
                                 (self.in_units, self.units))
        self.bias = self.param('bias', nn.initializers.zeros, self.units)

    def __call__(self, X):
        linear = jnp.matmul(X, self.weight) + self.bias
        return nn.relu(linear)

Next, we instantiate the `MyDense` class
and access its model parameters.


In [7]:
dense = MyDense(5, 3)
params = dense.init(d2l.get_key(), jnp.zeros((3, 5)))
params

FrozenDict({
    params: {
        weight: Array([[-0.05353878,  0.29904482,  0.04923093],
               [ 2.5594692 , -0.24971609,  0.33053744],
               [ 0.13617788, -0.03150544, -1.8140359 ],
               [-0.3789968 , -0.09270282, -0.6485172 ],
               [ 0.4420657 ,  0.52118987, -0.7718846 ]], dtype=float32),
        bias: Array([0., 0., 0.], dtype=float32),
    },
})

We can [**directly carry out forward propagation calculations using custom layers.**]


In [8]:
dense.apply(params, jax.random.uniform(d2l.get_key(),
                                       (2, 5)))

Array([[2.517842  , 0.        , 0.        ],
       [0.8743643 , 0.40055126, 0.        ]], dtype=float32)

We can also (**construct models using custom layers.**)
Once we have that we can use it just like the built-in fully connected layer.


In [9]:
net = nn.Sequential([MyDense(64, 8), MyDense(8, 1)])
Y, _ = net.init_with_output(d2l.get_key(), jax.random.uniform(d2l.get_key(),
                                                              (2, 64)))
Y

Array([[0.],
       [0.]], dtype=float32)

## Summary

We can design custom layers via the basic layer class. This allows us to define flexible new layers that behave differently from any existing layers in the library.
Once defined, custom layers can be invoked in arbitrary contexts and architectures.
Layers can have local parameters, which can be created through built-in functions.


## Exercises

1. Design a layer that takes an input and computes a tensor reduction,
   i.e., it returns $y_k = \sum_{i, j} W_{ijk} x_i x_j$.
1. Design a layer that returns the leading half of the Fourier coefficients of the data.


[Discussions](https://discuss.d2l.ai/t/17993)
